# Model Performance Testing
Load all trained models and evaluate their performance on the benchmark test set.

In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
import json
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
from sklearn.model_selection import train_test_split

In [ ]:
# Paths
dataset_path = "predictive_maintenance.csv"
xgb_path = "xgboost_model/xgb_model.joblib"
rf_path = "random_forest_model/rf_model.joblib"
lr_path = "logistic_regression_model/lr_model.joblib"
scaler_path = "logistic_regression_model/scaler.joblib"

In [ ]:
print("Loading models...")
xgb_model = joblib.load(xgb_path)
rf_model = joblib.load(rf_path)
lr_model = joblib.load(lr_path)
lr_scaler = joblib.load(scaler_path)

In [ ]:
print("Loading and preprocessing data...")
df = pd.read_csv(dataset_path)

# Feature Engineering
df['Temperature_Difference'] = df['Process temperature [K]'] - df['Air temperature [K]']
df['Power'] = df['Rotational speed [rpm]'] * df['Torque [Nm]']

df = df.rename(columns={
    'Air temperature [K]': 'Air_temperature_K',
    'Process temperature [K]': 'Process_temperature_K',
    'Rotational speed [rpm]': 'Rotational_speed_rpm',
    'Torque [Nm]': 'Torque_Nm',
    'Tool wear [min]': 'Tool_wear_min'
})

X = df.drop(['UDI', 'Product ID', 'Target', 'Failure Type'], axis=1)
y = df['Target']

X['Type'] = X['Type'].map({'L': 0, 'M': 1, 'H': 2})

_, X_test, _, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
print("Running predictions...")
y_pred_xgb = xgb_model.predict(X_test)
y_pred_rf = rf_model.predict(X_test)
X_test_scaled = lr_scaler.transform(X_test)
y_pred_lr = lr_model.predict(X_test_scaled)

In [ ]:
def print_report(name, y_true, y_pred):
    print(f"\n--- {name} Report ---")
    print(f"Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(classification_report(y_true, y_pred))

print_report("XGBoost", y_test, y_pred_xgb)
print_report("Random Forest", y_test, y_pred_rf)
print_report("Logistic Regression", y_test, y_pred_lr)